# Checkpoint analysis

The purpose of this notebook is to analyse the results of different checkpointing methods for extraction of the best performing model evaluated by the CUAD metrics without running the test framework on each checkpoint.

The problem with doing that is
* It takes 8h to run the test on a single GPU
* 10 epochs generates 20 checkpoints
* It cannot currently utilize multi-core 

It's simply not feasible

Thus experiments has been run to check if we can use the valloss as a proxy of a makeshift precision

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Reading in the data from WANDB. Notes on the different runs. The number below relates to the postfix ing model_name. All has been run on the associated paper infrastructure

* 1 - Min step both

* 2  - Min val loss

* 3 - 2.nd best tie min val loss 

* 4 - 3. best tie val_loss, Max precision

* 5 - 2.nd best max precision

* 6 - Third best max precision

* 7 -  Random control 1

* 8 - Random control 2


In [77]:
df_raw=pd.read_csv('./NotebookData/wandb_export_cuad_training_checkpoint_performance_analysis_test.csv')

In [78]:
performance_columns = [x for x in df_raw.columns if "performance" in x]
normal_columns = ['epoch','resume_from_pl_checkpoint','args/model_name',"performance_AUPR_testaupr"]

In [79]:
def get_checkpoint_values(checkpoint_name:str):
    splt = checkpoint_name.split('dataset-type_')[1].split('-')
    val_loss, precision, tp, global_step = 0, 0, 0, 0
    for i in splt:
        if 'val_loss' in i:
            val_loss = i.split('val_loss')[1]
        if 'precision' in i:
            precision = i.split('precision')[1]
        if 'tp' in i:
            tp = i.split('tp')[1]
        if 'global_step' in i:
            global_step = i.split('global_step')[1].split('.ckpt')[0]
    return val_loss, precision, tp, global_step

In [88]:
# Filter the data
df_raw['test']=df_raw['resume_from_pl_checkpoint'].apply(lambda x: get_checkpoint_values(x))
df_raw = df_raw.assign(**pd.DataFrame(df_raw.test.tolist(), columns=['val_loss','precision','tp','global_step']))
df_raw.drop(columns=['test'], inplace=True)

In [90]:
df_raw['val_loss'] = pd.to_numeric(df_raw['val_loss'])
df_raw['precision'] = pd.to_numeric(df_raw['precision'])
df_raw['tp'] = pd.to_numeric(df_raw['tp'])
df_raw['global_step'] = pd.to_numeric(df_raw['global_step'])
extra_columns = ["val_loss","precision","tp","global_step"]

In [91]:
df_raw.sort_values(by=['val_loss'], inplace=True)
df_final_small = df_raw[normal_columns + extra_columns]
df_final_small

,epoch,resume_from_pl_checkpoint,args/model_name,performance_AUPR_testaupr,val_loss,precision,tp,global_step
0,0,./out/checkpoints/checkpoint-val_loss-name_dat...,roberta_base_untrained_checkpointing_exp_test_8,0.332122,0.09,79.95,315.0,1680
1,2,./out/checkpoints/checkpoint-val_loss-name_dat...,roberta_base_untrained_checkpointing_exp_test_7,0.344022,0.09,80.37,307.0,960
3,5,./out/checkpoints/checkpoint-val_loss-name_dat...,roberta_base_untrained_checkpointing_exp_test_6,0.352360,0.12,80.49,326.0,2880
4,4,./out/checkpoints/checkpoint-val_loss-name_dat...,roberta_base_untrained_checkpointing_exp_test_4,0.360785,0.12,80.75,323.0,2400
2,3,./out/checkpoints/checkpoint-val_loss-name_dat...,roberta_base_untrained_checkpointing_exp_test_5,0.365151,0.12,78.85,302.0,480
5,3,./out/checkpoints/checkpoint-val_loss-name_dat...,roberta_base_untrained_checkpointing_exp_test_3,0.344011,0.13,79.35,319.0,1200
6,1,./out/checkpoints/checkpoint-val_loss-name_dat...,roberta_base_untrained_checkpointing_exp_test_2,0.360437,0.14,80.49,330.0,1920
7,0,./out/checkpoints/checkpoint-val_loss-name_dat...,roberta_base_untrained_checkpointing_exp_test_1,0.309513,0.25,75.38,294.0,240


In [92]:
df_final_small.corr()

,epoch,performance_AUPR_testaupr,val_loss,precision,tp,global_step
epoch,1.000000,0.644642,-0.361828,0.490016,0.423325,0.544045
performance_AUPR_testaupr,0.644642,1.000000,-0.614366,0.744011,0.577080,0.454076
val_loss,-0.361828,-0.614366,1.000000,-0.881162,-0.473027,-0.443374
precision,0.490016,0.744011,-0.881162,1.000000,0.793256,0.738196
tp,0.423325,0.577080,-0.473027,0.793256,1.000000,0.885179
global_step,0.544045,0.454076,-0.443374,0.738196,0.885179,1.000000


In [93]:
corr = df_final_small.corr()
corr.style.background_gradient(cmap='coolwarm')


,epoch,performance_AUPR_testaupr,val_loss,precision,tp,global_step
epoch,1.000000,0.644642,-0.361828,0.490016,0.423325,0.544045
performance_AUPR_testaupr,0.644642,1.000000,-0.614366,0.744011,0.577080,0.454076
val_loss,-0.361828,-0.614366,1.000000,-0.881162,-0.473027,-0.443374
precision,0.490016,0.744011,-0.881162,1.000000,0.793256,0.738196
tp,0.423325,0.577080,-0.473027,0.793256,1.000000,0.885179
global_step,0.544045,0.454076,-0.443374,0.738196,0.885179,1.000000


Pulling in the other performance measures

In [101]:
df_final_large = df_raw[list(set(normal_columns + performance_columns + extra_columns))]

# drop the columns with NaN and 0 values
df_final_large.dropna(axis=1, inplace=True)
df_final_large = df_final_large.loc[:, (df_final_large != 0).any(axis=0)]
# drop some columns
df_final_large.drop(columns=['performance_stats_testNoAns_total','performance_stats_testHasAns_total','performance_stats_testtotal'], inplace=True)
df_final_large

/var/folders/nt/g2ndh3nx4dj16m34wq3h41lh0000gn/T/ipykernel_85271/358290906.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_large.dropna(axis=1, inplace=True)
/var/folders/nt/g2ndh3nx4dj16m34wq3h41lh0000gn/T/ipykernel_85271/358290906.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_large.drop(columns=['performance_stats_testNoAns_total','performance_stats_testHasAns_total','performance_stats_testtotal'], inplace=True)


,resume_from_pl_checkpoint,global_step,performance_stats_testexact,performance_stats_testNoAns_exact,val_loss,performance_stats_testHasAns_exact,args/model_name,performance_AUPR_testaupr,performance_stats_testNoAns_f1,epoch,performance_stats_testbest_exact,performance_stats_testf1,precision,performance_stats_testbest_f1,tp,performance_stats_testHasAns_f1
0,./out/checkpoints/checkpoint-val_loss-name_dat...,1680,74.988045,79.475838,0.09,64.389069,roberta_base_untrained_checkpointing_exp_test_8,0.332122,79.475838,0,74.988045,78.262405,79.95,78.262405,315.0,75.396614
1,./out/checkpoints/checkpoint-val_loss-name_dat...,960,70.994736,73.076920,0.09,66.077171,roberta_base_untrained_checkpointing_exp_test_7,0.344022,73.076920,2,71.664276,75.075485,80.37,75.270790,307.0,79.795555
3,./out/checkpoints/checkpoint-val_loss-name_dat...,2880,74.772835,77.671883,0.12,67.926048,roberta_base_untrained_checkpointing_exp_test_6,0.352360,77.671883,5,74.820663,78.517235,80.49,78.521896,326.0,80.513718
4,./out/checkpoints/checkpoint-val_loss-name_dat...,2400,75.179337,77.535736,0.12,69.614151,roberta_base_untrained_checkpointing_exp_test_4,0.360785,77.535736,4,75.179337,78.800674,80.75,78.781425,323.0,81.788109
2,./out/checkpoints/checkpoint-val_loss-name_dat...,480,69.966522,71.170868,0.12,67.122185,roberta_base_untrained_checkpointing_exp_test_5,0.365151,71.170868,3,70.994736,73.702301,78.85,73.843689,302.0,79.680893
5,./out/checkpoints/checkpoint-val_loss-name_dat...,1200,77.953133,82.062630,0.13,68.247589,roberta_base_untrained_checkpointing_exp_test_3,0.344011,82.062630,3,77.953133,81.189377,79.35,81.189377,319.0,79.126984
6,./out/checkpoints/checkpoint-val_loss-name_dat...,1920,76.255379,80.973450,0.14,65.112541,roberta_base_untrained_checkpointing_exp_test_2,0.360437,80.973450,1,76.351028,79.827324,80.49,79.861877,330.0,77.120468
7,./out/checkpoints/checkpoint-val_loss-name_dat...,240,62.051651,63.614704,0.25,58.360130,roberta_base_untrained_checkpointing_exp_test_1,0.309513,63.614704,0,70.253464,66.311935,75.38,70.405693,294.0,72.682083


In [102]:
corr = df_final_large.corr()
corr.style.background_gradient(cmap='coolwarm')

,global_step,performance_stats_testexact,performance_stats_testNoAns_exact,val_loss,performance_stats_testHasAns_exact,performance_AUPR_testaupr,performance_stats_testNoAns_f1,epoch,performance_stats_testbest_exact,performance_stats_testf1,precision,performance_stats_testbest_f1,tp,performance_stats_testHasAns_f1
global_step,1.000000,0.691579,0.674045,-0.443374,0.579436,0.454076,0.674045,0.544045,0.635220,0.703740,0.738196,0.710581,0.885179,0.505201
performance_stats_testexact,0.691579,1.000000,0.988628,-0.715677,0.779620,0.611231,0.988628,0.411268,0.905220,0.999228,0.846931,0.982617,0.878248,0.565264
performance_stats_testNoAns_exact,0.674045,0.988628,1.000000,-0.660754,0.676575,0.519714,1.000000,0.289502,0.934461,0.985690,0.799246,0.987085,0.882703,0.438292
val_loss,-0.443374,-0.715677,-0.660754,1.000000,-0.752780,-0.614366,-0.660754,-0.361828,-0.372791,-0.720127,-0.881162,-0.589231,-0.473027,-0.656610
performance_stats_testHasAns_exact,0.579436,0.779620,0.676575,-0.752780,1.000000,0.828686,0.676575,0.808230,0.541089,0.788069,0.818751,0.700929,0.624555,0.942678
performance_AUPR_testaupr,0.454076,0.611231,0.519714,-0.614366,0.828686,1.000000,0.519714,0.644642,0.336336,0.622614,0.744011,0.511490,0.577080,0.816126
performance_stats_testNoAns_f1,0.674045,0.988628,1.000000,-0.660754,0.676575,0.519714,1.000000,0.289502,0.934461,0.985690,0.799246,0.987085,0.882703,0.438292
epoch,0.544045,0.411268,0.289502,-0.361828,0.808230,0.644642,0.289502,1.000000,0.252394,0.427521,0.490016,0.369638,0.423325,0.884925
performance_stats_testbest_exact,0.635220,0.905220,0.934461,-0.372791,0.541089,0.336336,0.934461,0.252394,1.000000,0.899389,0.583909,0.963832,0.856564,0.293855
performance_stats_testf1,0.703740,0.999228,0.985690,-0.720127,0.788069,0.622614,0.985690,0.427521,0.899389,1.000000,0.860661,0.981949,0.887017,0.583533
